# Historic Admissions Data

In [1]:
import pandas as pd
import datetime as dt
from datetime import date
from pathlib import Path

In [2]:
today = date.today()
today_str = today.strftime("%Y%m%d")
print(today)
print(today_str)


2020-12-16
20201216


In [3]:
if today<(dt.date(today.year, 9, 1)):
    acad_yr = today.year
else:
    acad_yr = today.year + 1

print(f"acad_yr={acad_yr:d}")

acad_yr=2021


In [4]:
#data_path = Path(r"C:\JW\Python\Admissions\funnel")
data_path = Path(r"\\psc-data\E\Applications\Admissions\funnel")
data_store = data_path / "data/stage_data"

df = pd.read_hdf(data_store, key="weekly", mode='r')
df = df[(df["year_term"] > "2011.Spring")]


In [5]:
b_yr = 2012
e_yr = 2021
term = 'Fall'

terms = [f"{y}.{term}" for y in range(b_yr, e_yr+1)]
print(terms)
stages = [ 'Applied', 'Accepted', 'Deposited']


['2012.Fall', '2013.Fall', '2014.Fall', '2015.Fall', '2016.Fall', '2017.Fall', '2018.Fall', '2019.Fall', '2020.Fall', '2021.Fall']


In [6]:
summ = df.groupby(["year_term", "stage"]).sum()
summ_t = summ.transpose()


In [7]:
print(summ.shape)
summ.head()


(60, 54)


00  01  02  03  04  05  06  07  08  09  ...   44   45  \
year_term   stage                                              ...             
2012.Fall   Accepted    0   1   1   1   1   1   1   1   2   2  ...  416  413   
            Applied     0   1   1   1   2   5   6   7  11  14  ...  582  580   
            Deposited   0   3   3   3   3   3   3   3   4   4  ...  368  368   
2012.Spring Accepted    0   0   0   0   0   0   0   0   0   0  ...    1    1   
            Applied     0   0   0   0   0   1   1   1   1   1  ...    2    2   

                        46   47   48   49   50   51   52   53  
year_term   stage                                              
2012.Fall   Accepted   396  395  359  360  343  290  276  276  
            Applied    559  554  506  507  490  433  420  420  
            Deposited  369  369  372  372  373  365  365  365  
2012.Spring Accepted     1    1    1    1    1    1    1    1  
            Applied      2    2    2    2    2    2    2    2  

[5 rows x 54 columns]

In [8]:
summ_t.head()


year_term 2012.Fall                   2012.Spring                   2013.Fall  \
stage      Accepted Applied Deposited    Accepted Applied Deposited  Accepted   
00                0       0         0           0       0         0         0   
01                1       1         3           0       0         0        18   
02                1       1         3           0       0         0        20   
03                1       1         3           0       0         0        23   
04                1       2         3           0       0         0        24   

year_term                   2013.Spring  ... 2020.Fall 2020.Spring          \
stage     Applied Deposited    Accepted  ... Deposited    Accepted Applied   
00              0         0           0  ...         0           0       0   
01             83         9           8  ...        10           2       5   
02             73         9           8  ...        10           2       8   
03             77        10           8  ...        10           3      10   
04             93        13          13  ...        10           3      12   

year_term           2021.Fall                   2021.Spring                    
stage     Deposited  Accepted Applied Deposited    Accepted Applied Deposited  
00                0         0       0         0           0       0         0  
01               11        24      46        18          20      42        17  
02               12        24      50        16          20      45        17  
03               11        23      56        16          22      50        17  
04               11        23      63        17          24      50        19  

[5 rows x 60 columns]

In [9]:
def adm_week(d):
    """
    returns calendar week number and Admissions Week Number for a given date, d
    """
    year = d.year
    if d >= date(year, 9, 1):
        adm_year_start = year
    else:
        adm_year_start = year - 1

    week_number = d.isocalendar()[1]
    adm_start_week = date(adm_year_start, 9, 1).isocalendar()[1]

    if week_number >= adm_start_week:
        adm_week_number = (week_number - adm_start_week) + 1
    else:
        adm_week_number = 53 + (week_number - adm_start_week)

    return (week_number, adm_week_number)


week_number, adm_week_number = adm_week(today)
print(week_number, adm_week_number)


51 16


In [10]:
def adm_week_end_date(awn, acad_yr):
    """
    returns calendar date for the Friday of the given Admissions Week Number, awn
        and academic year, acad_yr
    """
    
    adm_start_date = dt.date((acad_yr-1), 9, 1)
    adm_start_week = adm_start_date.isocalendar()[1]
    
    if awn <= (53 - adm_start_week):
        wn = adm_start_week + awn - 1
        year = acad_yr - 1
    else:
        wn = awn - (53 - adm_start_week) - 1
        year = acad_yr

    week_end = dt.datetime.strptime(f'{year}-{wn}-1', "%Y-%W-%w").date() + dt.timedelta(days=6.9)
    
    return(week_end)

print(today, adm_week_end_date(adm_week_number, acad_yr))

2020-12-16 2020-12-27


In [ ]:
week = adm_week(today)[1]
print(week, f"{week:02d}")
week = f"{week:02d}"


In [ ]:
stg = "Deposited"
for t in terms:
    print(t, str(week), summ.loc[(t, stg)][str(week)])

print(terms[-1], terms[-2], 
    summ.loc[(terms[-1], stg)][str(week)] - summ.loc[(terms[-2], stg)][str(week)]
)


In [ ]:
day2 = date(acad_yr, 5, 1)
print(day2)
week2 = adm_week(day2)[1]
print(week2)

for t in reversed(terms):
    print(t, str(week2), summ.loc[(t, stg)][str(week2)])

for t in reversed(terms):
    print(summ.loc[(t, stg)][str(week2)])


In [ ]:
day2 = date(acad_yr, 8, 25)
print(day2)
week2 = adm_week(day2)[1]
print(week2)
week2 = f"{week2:02d}"


for t in reversed(terms):
    print(t, str(week2), summ.loc[(t, stg)][str(week2)])

for t in reversed(terms):
    print(summ.loc[(t, stg)][str(week2)])


In [ ]:
print("Now to end of year")
for t in reversed(terms):
    print(t, str(week), summ.loc[(t, stg)][str(week)],
        "52", summ.loc[(t, stg)]["52"],
        summ.loc[(t, stg)]["52"] - summ.loc[(t, stg)][str(week)]
    )


In [ ]:
weeks = range(1, 53)
for w in weeks:
    f = f"{w:02d}"
    yet_to_deposit = (
        summ.loc[("2018.Fall", "Deposited")]["52"]
        - summ.loc[("2018.Fall", "Deposited")][f]
    )
    print(w, yet_to_deposit)


In [ ]:
summ.describe()


In [ ]:
summ.index


In [ ]:
# Goal
GOAL = 260
summ.loc[("2019.Fall", "Deposited")]["52"] = GOAL


In [ ]:
df1 = summ.copy()
weeks = range(1, 53)
for w in weeks:
    f = f"{w:02d}"
    df1[f] = summ.loc[:]["52"] - summ.loc[:][f]

print(df1.shape)


In [ ]:
df1.head()


In [ ]:
df1_t = df1.transpose()


In [ ]:
df1_t.head()


In [ ]:
# from bokeh.layouts import widgetbox, row
# from bokeh.models.widgets import MultiSelect, RadioGroup, Select
# from bokeh.plotting import figure, curdoc
from bokeh.plotting import figure, output_file, show

# from bokeh.palettes import Blues9
from bokeh.palettes import Set2_8
from bokeh.io import output_notebook


In [ ]:
output_notebook()
output_file("DepositsToEndOFYear.html")


In [ ]:
TOOLS = "pan,wheel_zoom,box_zoom,save,reset"


In [ ]:
stage = "Deposited"
term = "2019.Fall"

all_terms = sorted(list(df["year_term"].dropna().unique()))
all_terms = [l for l in all_terms if "Fall" in l]
all_terms.remove(term)
# all_terms.remove('2013.Fall')
all_terms.remove("2012.Fall")
all_terms.reverse()
y_max = df1_t[(term, stage)].max()
y_min = df1_t[(term, stage)].min()
for t in all_terms:
    ymx = df1_t[(t, stage)].max()
    ymn = df1_t[(t, stage)].min()
    if ymx > y_max:
        y_max = ymx
    if ymn < y_min:
        y_min = ymn

print(term, stage, y_min, y_max)


In [ ]:
print(all_terms)


In [ ]:
title = f"'{stage}' Gained to End of Year,  Goal={GOAL}  (Week {adm_week_number:d})"


In [ ]:
p = figure(
    plot_width=800,
    plot_height=600,
    title=title,
    x_axis_label="Admissions Week Number (year starts Sept 1)",
    y_axis_label=stage,
    tools=TOOLS,
    x_range=(0, 54),
    y_range=(y_min * 1.05, y_max * 1.05),
)

c = 0
for t in all_terms:
    # p.line(df1_t.index, df1_t[(t, stage)], color=Blues9[c], legend=t)
    p.line(df1_t.index, df1_t[(t, stage)], color=Set2_8[c], legend=t)
    if c <= 7:
        c += 1

p.line(df1_t.index, df1_t[(term, stage)], color="red", line_width=2, legend=term)

# week_number line
p.line(
    (adm_week_number, adm_week_number),
    (-1000, 5000),
    color="green",
    line_width=0.8,
    line_dash="dashed",
    legend=f"Week {adm_week_number:d}",
    alpha=0.8,
)

p.legend.location = "top_right"

show(p)


In [ ]:
# get list of 2019.Fall applicants
dep = (
    df.loc[
        (df["year_term"] == "2019.Fall") & (df["stage"] == "Deposited") & (df["53"] > 0)
    ]
    .sort_values(["PEOPLE_CODE_ID"])
    .drop_duplicates(["PEOPLE_CODE_ID"])
)
print(dep.shape)


In [ ]:
dep.head()


In [ ]:
dep.to_csv("2019-Fall-Deposited.csv")


In [ ]:
df.loc[(df["PEOPLE_CODE_ID"] == "P000053667")]


In [ ]:
# melt
import numpy as np

a = [
    summ.loc[("2017.Fall", "Deposited")]["53"]
    - summ.loc[("2017.Fall", "Deposited")][str(week)],
    summ.loc[("2016.Fall", "Deposited")]["53"]
    - summ.loc[("2016.Fall", "Deposited")][str(week)],
    summ.loc[("2015.Fall", "Deposited")]["53"]
    - summ.loc[("2015.Fall", "Deposited")][str(week)],
    summ.loc[("2014.Fall", "Deposited")]["53"]
    - summ.loc[("2014.Fall", "Deposited")][str(week)],
    summ.loc[("2013.Fall", "Deposited")]["53"]
    - summ.loc[("2013.Fall", "Deposited")][str(week)],
    summ.loc[("2012.Fall", "Deposited")]["53"]
    - summ.loc[("2012.Fall", "Deposited")][str(week)],
]
print(a)
print(sum(a))
print(sum(a) / 6)
print(np.mean(a))


In [ ]:
for st in stages:
    df = summ_t.loc[:, (terms, st)]
    #print(df.head())
    

In [ ]:
df

In [11]:
summ = df.groupby(["year_term", "stage"]).sum()
summ_t = summ.transpose()
summ_t = summ_t.reset_index().rename(columns={"index": "week_num"})
summ_t["wn"] = summ_t.loc[:, ["week_num"]].astype(int)
summ_t.head()

year_term week_num 2012.Fall                   2012.Spring                    \
stage               Accepted Applied Deposited    Accepted Applied Deposited   
0               00         0       0         0           0       0         0   
1               01         1       1         3           0       0         0   
2               02         1       1         3           0       0         0   
3               03         1       1         3           0       0         0   
4               04         1       2         3           0       0         0   

year_term 2013.Fall                    ... 2020.Spring                    \
stage      Accepted Applied Deposited  ...    Accepted Applied Deposited   
0                 0       0         0  ...           0       0         0   
1                18      83         9  ...           2       5        11   
2                20      73         9  ...           2       8        12   
3                23      77        10  ...           3      10        11   
4                24      93        13  ...           3      12        11   

year_term 2021.Fall                   2021.Spring                   wn  
stage      Accepted Applied Deposited    Accepted Applied Deposited     
0                 0       0         0           0       0         0  0  
1                24      46        18          20      42        17  1  
2                24      50        16          20      45        17  2  
3                23      56        16          22      50        17  3  
4                23      63        17          24      50        19  4  

[5 rows x 62 columns]

In [12]:
week_end = (
    lambda r: adm_week_end_date(int(r["wn"]), acad_yr)
)
summ_t["Week_End"] = summ_t.apply(week_end, axis=1)


In [13]:
summ_t = summ_t.drop(columns=["wn", "week_num"], axis=1).set_index("Week_End")

C:\Users\jwalton\.conda\envs\py38jl\lib\site-packages\pandas\core\generic.py:3878: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [14]:
summ_t.head()

year_term  2012.Fall                   2012.Spring                    \
stage       Accepted Applied Deposited    Accepted Applied Deposited   
Week_End                                                               
2020-09-06         0       0         0           0       0         0   
2020-09-13         1       1         3           0       0         0   
2020-09-20         1       1         3           0       0         0   
2020-09-27         1       1         3           0       0         0   
2020-10-04         1       2         3           0       0         0   

year_term  2013.Fall                   2013.Spring  ... 2020.Fall 2020.Spring  \
stage       Accepted Applied Deposited    Accepted  ... Deposited    Accepted   
Week_End                                            ...                         
2020-09-06         0       0         0           0  ...         0           0   
2020-09-13        18      83         9           8  ...        10           2   
2020-09-20        20      73         9           8  ...        10           2   
2020-09-27        23      77        10           8  ...        10           3   
2020-10-04        24      93        13          13  ...        10           3   

year_term                    2021.Fall                   2021.Spring          \
stage      Applied Deposited  Accepted Applied Deposited    Accepted Applied   
Week_End                                                                       
2020-09-06       0         0         0       0         0           0       0   
2020-09-13       5        11        24      46        18          20      42   
2020-09-20       8        12        24      50        16          20      45   
2020-09-27      10        11        23      56        16          22      50   
2020-10-04      12        11        23      63        17          24      50   

year_term             
stage      Deposited  
Week_End              
2020-09-06         0  
2020-09-13        17  
2020-09-20        17  
2020-09-27        17  
2020-10-04        19  

[5 rows x 60 columns]

In [15]:
print(today_str)
writer = pd.ExcelWriter(
    f"{term}_HistoricAdmissionsData_{b_yr:d}-{e_yr:d}_{today_str}.xlsx"
)
for st in stages:
    df_sumt = summ_t.loc[:, (terms, st)]
    #df_sumt.T.reset_index().T.to_excel(writer, st)
    df_sumt.to_excel(writer, st)

writer.save()


20201216
